## Importação das bibliotecas

In [1]:
import pandas as pd

## Funções

### Função de sequeciamento

In [2]:
def sequencing(df, field_to_sequence, window, fields_to_group:list, fields_to_sort):
    if field_to_sequence not in list(df):
        print('field', field_to_sequence, 'is not in dataframe')
    for f in fields_to_group:
        if f not in list(df):
            print('field', f, 'is not in dataframe')
    return df.sort_values(
            fields_to_sort
        ).assign(
        **{'Seq_{field}'.format(field=field_to_sequence): lambda x: pd.concat(
            [
                dpg.sum()
                for dpg in x[
                    fields_to_group + [field_to_sequence]
                ].groupby(
                    by=fields_to_group
                )[[field_to_sequence]].rolling(window)
            ]
        ).values}
    )

### Função de sequeciamento para dicionario

In [3]:
def seq_to_dict(seq, field_name):
    return {field_name+str(k):c for k, c in enumerate(list(seq)[::-1])}

## Criação dos dataframes

### Carga dos jogos

In [4]:
game_file = 'games_9.parquet'


df = pd.read_parquet('games_9.parquet').query("`Camp.` == 'Premier League'").reset_index(drop=True)
df_hist = df.query("Resultado != ''").reset_index(drop=True)

### Rodadas do ultimo ano

In [5]:
df_rodada = df_hist.query(
    "Ano == '{ano}'".format(
        ano = df_hist['Ano'].max()
    )
).assign(
    **{
        'rodada_text': lambda x: x['Rodada'].apply(
            lambda y: ' '.join(y.split(' ')[:-1])
        )
        , 'rodada_number': lambda x: x['Rodada'].apply(
            lambda y: int(y.split(' ')[-1])
        )
    }
)[
    ['Rodada', 'rodada_text', 'rodada_number']
]

### Jogos da próxima rodada

In [6]:
df_next_round = df.query(
    "Ano == '{ano}'".format(
        ano = df_hist['Ano'].max()
    )
    + " and Rodada == '{rodada_text} {rodada_number}'".format(
        rodada_text = df_rodada['rodada_text'].max()
        , rodada_number = df_rodada['rodada_number'].max()+1
    )
)

## Etapa 1

### Tratamento

<ul>
    <li> Retirar partidas Canceladas </li>
    <li> Retirar partidas adiadas </li>
    <li> Seleção dos campos </li>
    <ul>
        <li> <b>Resultado</b> - Resultado da partida</li>
        <li> <b>fgk_equipe</b> - código da equipe</li>
        <li> <b>Ano</b> - Ano do campeonato</li>
        <li> <b>Rodada</b> - Rodada do campeonato</li>
        <li> <b>Local</b> - Local da partida</li>
        <li> <b>GP</b> - Gols a favor da partida</li>
        <li> <b>GC</b> - Gols contra da partida</li>
        <li> <b>Oponente</b> - código do oponente</li>
        <li> <b>id</b> - código da partida</li> 
    </ul>
    <li> Tratamento da "Rodada" para conter apenas números </li>
    <li> Ordenação do dataframe por: fgk_equipe, Ano, Rodada </li>
    <li> Tratamento do "Local" substituindo: "Visitante" por 0 e "Em cada" por 1 </li>
</ul>

In [7]:
df_1 = df_hist.query(
    "not Notas.str.contains('Partida Cancelada')" # Retira partida Cancelada
    + " and not Notas.str.contains('Partida adiada')" # Retirada partida adiada
    , engine='python'
)[
    ['Resultado', 'fgk_equipe', 'Ano', 'Rodada', 'Local', 'GP', 'GC', 'Oponente', 'id']
].assign(
    **{'Rodada': lambda x: x['Rodada'].apply(
        lambda y: int(y.split(' ')[-1])
    )}
).sort_values(
    ['fgk_equipe', 'Ano', 'Rodada']
).replace(
    {'Local':{'Visitante':'0', 'Em casa':'1'}}
)
df_1

,Resultado,fgk_equipe,Ano,Rodada,Local,GP,GC,Oponente,id
304,D,17892952,2014-2015,1,1,0,1,8602292d,0a235dd1
305,E,17892952,2014-2015,2,0,1,1,bd8769d1,ee81980a
306,V,17892952,2014-2015,3,0,1,0,b8fd03ef,9e73f0ce
307,D,17892952,2014-2015,4,1,0,1,a2d435b3,238517ea
308,E,17892952,2014-2015,5,0,2,2,a757999c,e8592bc3
...,...,...,...,...,...,...,...,...,...
6204,V,fd962109,2022-2023,12,1,3,0,8602292d,e6102606
6205,V,fd962109,2022-2023,13,0,3,2,5bfb9659,d7b30e2b
6206,E,fd962109,2022-2023,14,1,0,0,d3fd31cc,9c4402e1
6207,D,fd962109,2022-2023,15,0,1,2,b8fd03ef,6ef27b3c


### Verificação de NA

In [8]:
df_1.isna().sum()

Resultado     0
fgk_equipe    0
Ano           0
Rodada        0
Local         0
GP            0
GC            0
Oponente      0
id            0
dtype: int64

## Etapa 2

### Tratamento
<ul>
    <li> "parm" definie os parametro padrão </li>
    <ul>
        <li> <b>window</b> - 5 </li>
        <li> <b>fields_to_group</b> - fgk_equipe e Ano </li>
        <li> <b>fields_to_sort</b> - fgk_equipe, Ano e Rodada </li>
    </ul>
    <li> Sequenciamento do "Resultado" em "Seq_Resultado"</li>
    <li> Sequenciamento do "Local" em "Seq_Local" </li>
    <li> Sequenciamento do "GP" em "Seq_GP" </li>
    <li> Sequenciamento do "GC" em "Seq_GC" </li>
    <li> Tratamento do "Seq_Resultado" substituindo: "V" por 2, "E" por 1 e "D" por 0</li>
    <li> Tratamento de "SSeq_Local", "Seq_GP" e "Seq_GC" preenchendo com zerofill(5)</li>
</ul>

In [9]:
parms = {'window': 5, 'fields_to_group': ['fgk_equipe', 'Ano'], 'fields_to_sort':['fgk_equipe', 'Ano', 'Rodada']}

# Resultado
df_1_temp = sequencing(
    **{'df': df_1, 'field_to_sequence': 'Resultado', **parms}
)

# Local
df_1_temp = sequencing(
    **{'df': df_1_temp, 'field_to_sequence': 'Local', **parms}
)

# GP
df_1_temp = sequencing(
    **{'df': df_1_temp, 'field_to_sequence': 'GP', **parms}
)

# GC
df_2 = sequencing(
    **{'df': df_1_temp, 'field_to_sequence': 'GC', **parms}
)

del df_1_temp

df_2 = df_2.assign(
    **{
        'Seq_Resultado': lambda x: x['Seq_Resultado'].str.replace('V', '2').str.replace('E', '1').str.replace('D', '0')
        , 'Seq_Local': lambda x: x['Seq_Local'].astype('int').astype('str').str.zfill(5)
        , 'Seq_GP': lambda x: x['Seq_GP'].astype('int').astype('str').str.zfill(5)
        , 'Seq_GC': lambda x: x['Seq_GC'].astype('int').astype('str').str.zfill(5)
    }
)

df_2

,Resultado,fgk_equipe,Ano,Rodada,Local,GP,GC,Oponente,id,Seq_Resultado,Seq_Local,Seq_GP,Seq_GC
304,D,17892952,2014-2015,1,1,0,1,8602292d,0a235dd1,0,00001,00000,00001
305,E,17892952,2014-2015,2,0,1,1,bd8769d1,ee81980a,01,00010,00001,00011
306,V,17892952,2014-2015,3,0,1,0,b8fd03ef,9e73f0ce,012,00100,00011,00110
307,D,17892952,2014-2015,4,1,0,1,a2d435b3,238517ea,0120,01001,00110,01101
308,E,17892952,2014-2015,5,0,2,2,a757999c,e8592bc3,01201,10010,01102,11012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6204,V,fd962109,2022-2023,12,1,3,0,8602292d,e6102606,20012,01011,31123,24320
6205,V,fd962109,2022-2023,13,0,3,2,5bfb9659,d7b30e2b,00122,10110,11233,43202
6206,E,fd962109,2022-2023,14,1,0,0,d3fd31cc,9c4402e1,01221,01101,12330,32020
6207,D,fd962109,2022-2023,15,0,1,2,b8fd03ef,6ef27b3c,12210,11010,23301,20202


### Verificação de NA

In [10]:
df_2.isna().sum()

Resultado        0
fgk_equipe       0
Ano              0
Rodada           0
Local            0
GP               0
GC               0
Oponente         0
id               0
Seq_Resultado    0
Seq_Local        0
Seq_GP           0
Seq_GC           0
dtype: int64

## Etapa 3 e 4

### Tratamento dos dados de treino
<ul>
    <li>Tratamento da "Rodada" somando +1</li>
    <li>Tratamento do dataframe com join para obter dados da próxima rodada</li>
    <ul>
        <li>Target</li>
        <li>Target_Oponente</li>
        <li>GL</li>
        <li>Target_GP</li>
        <li>Target_GC</li>
    </ul>
    <li>Tratamento do "Target" substituindo: "V" por 2, "E" por 1 e "D" por 0</li>
    <li>Filtro de Rodadas maiores que 5 e menores que 39</li>
</ul>

In [11]:
df_3 = df_2.assign(
    **{'Rodada': lambda x: x['Rodada']+1}
).set_index(
    ['Ano', 'fgk_equipe', 'Rodada']
).join(
    df_2.rename(
        columns={
            'Resultado':'Target'
            , 'Oponente':'Target_Oponente'
            , 'Local':'GL'
            , 'GP':'Target_GP'
            , 'GC':'Target_GC'
        }
    ).set_index(
        ['Ano', 'fgk_equipe', 'Rodada']
    )[['Target', 'GL','Target_Oponente', 'Target_GP', 'Target_GC']]
).assign(
    **{'Target': lambda x: x['Target'].str.replace('V', '2').str.replace('E', '1').str.replace('D', '0')} # Target
).query(
    "Rodada > 5 and Rodada < 39"
)

### Tratamento dos dados da próxima rodada

In [12]:
df_3_next_round = df_3.query(
    "Ano == '{ano}'".format(
        ano = df_hist['Ano'].max()
    )
    + " and Rodada == {rodada_number}".format(
        rodada_number = df_rodada['rodada_number'].max()+1
    )
)[list(df_3)[:-5]]

In [13]:
df_4_next_round = df_3_next_round.drop(
    columns=['Local', 'Oponente', 'id']
).join(
    df_next_round.replace(
        {'Local':{'Visitante':'0', 'Em casa':'1'}}
    ).assign(
        **{'Rodada': lambda x: x['Rodada'].apply(
            lambda y: int(y.split(' ')[-1])
        )}
    )[
        ['Ano', 'fgk_equipe', 'Rodada', 'id', 'Local', 'Oponente']
    ].set_index(['Ano', 'fgk_equipe', 'Rodada'])
).rename(
    columns={'Local': 'GL', 'Oponente': 'Target_Oponente'}
)

### Verificação de NA dados de treino

In [14]:
df_3.isna().sum()

Resultado           0
Local               0
GP                  0
GC                  0
Oponente            0
id                  0
Seq_Resultado       0
Seq_Local           0
Seq_GP              0
Seq_GC              0
Target             42
GL                 42
Target_Oponente    42
Target_GP          42
Target_GC          42
dtype: int64

### Correção dados treino

In [15]:
df_4 = df_3

# print(df_3.shape)
# df_4 = df_3[df_3['GL'].notna()]
# print(df_4.shape)

### Verificação de NA dados da próxima rodada

In [16]:
df_4_next_round.isna().sum()

Resultado          0
GP                 0
GC                 0
Seq_Resultado      0
Seq_Local          0
Seq_GP             0
Seq_GC             0
id                 0
GL                 0
Target_Oponente    0
dtype: int64

### Correção dados da próxima rodada

In [17]:
print(df_4_next_round.shape)
df_4_next_round = df_4_next_round[df_4_next_round['Resultado'].notna()]
print(df_4_next_round.shape)

(20, 10)
(20, 10)


## Etapa 5 e 6

### Tratamento dos dados de treino
<ul>
    <li> ... </li>
</ul>

In [18]:
df_5 = pd.DataFrame()

for r in df_4.iterrows():
    df_5 = pd.concat(
        [
            df_5
            , pd.Series({
                **dict(zip(['Ano', 'Equipe', 'Rodada'], r[0]))
                , **seq_to_dict(r[1]['Seq_Resultado'], 'TR')
                , **seq_to_dict(r[1]['Seq_Local'], 'TL')
                , **seq_to_dict(r[1]['Seq_GP'], 'TP')
                , **seq_to_dict(r[1]['Seq_GC'], 'TC')
                , **{
                    'TGL': r[1]['GL']
                    , 'Oponente': r[1]['Target_Oponente']
                    , 'Target': r[1]['Target']
                    , 'GP': r[1]['Target_GP']
                    , 'GC': r[1]['Target_GC']
                    , 'id': r[1]['id']
                }
            })
        ]
        , axis=1
    )

df_6 = df_5.T
df_6

,Ano,Equipe,Rodada,TR0,TR1,TR2,TR3,TR4,TL0,TL1,...,TC1,TC2,TC3,TC4,TGL,Oponente,Target,GP,GC,id
0,2014-2015,17892952,6,1,0,2,1,0,0,1,...,1,0,1,1,1,b2b47a98,2,1,0,e8592bc3
0,2014-2015,17892952,7,2,1,0,2,1,1,0,...,2,1,0,1,0,8ef52968,0,1,3,9d2c4411
0,2014-2015,17892952,8,0,2,1,0,2,0,1,...,0,2,1,0,1,fb10988f,2,2,1,bc4351a4
0,2014-2015,17892952,9,2,0,2,1,0,1,0,...,3,0,2,1,0,33c895d4,0,0,1,6fa0779f
0,2014-2015,17892952,10,0,2,0,2,1,0,1,...,1,3,0,2,1,7c21e445,1,2,2,2be4379e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-2023,fd962109,13,2,1,0,0,2,1,1,...,2,3,4,2,0,5bfb9659,2,3,2,e6102606
0,2022-2023,fd962109,14,2,2,1,0,0,0,1,...,0,2,3,4,1,d3fd31cc,1,0,0,d7b30e2b
0,2022-2023,fd962109,15,1,2,2,1,0,1,0,...,2,0,2,3,0,b8fd03ef,0,1,2,9c4402e1
0,2022-2023,fd962109,16,0,1,2,2,1,0,1,...,0,2,0,2,1,19538871,0,1,2,6ef27b3c


### Tratamento dos dados da próxima rodada

In [19]:
df_5_next_round = pd.DataFrame()

for r in df_4_next_round.iterrows():
    df_5_next_round = pd.concat(
        [
            df_5_next_round
            , pd.Series({
                **dict(zip(['Ano', 'Equipe', 'Rodada'], r[0]))
                , **seq_to_dict(r[1]['Seq_Resultado'], 'TR')
                , **seq_to_dict(r[1]['Seq_Local'], 'TL')
                , **seq_to_dict(r[1]['Seq_GP'], 'TP')
                , **seq_to_dict(r[1]['Seq_GC'], 'TC')
                , **{
                    'TGL': r[1]['GL']
                    , 'Oponente': r[1]['Target_Oponente']
#                     , 'Target': r[1]['Target']
#                     , 'GP': r[1]['Target_GP']
#                     , 'GC': r[1]['Target_GC']
                    , 'id': r[1]['id']
                }
            })
        ]
        , axis=1
    )

df_6_next_round = df_5_next_round.T
df_6_next_round

,Ano,Equipe,Rodada,TR0,TR1,TR2,TR3,TR4,TL0,TL1,...,TP3,TP4,TC0,TC1,TC2,TC3,TC4,TGL,Oponente,id
0,2022-2023,18bb7c10,17,2,2,2,1,2,0,0,...,1,1,0,0,0,1,0,1,7c21e445,18bb7c10
0,2022-2023,19538871,17,2,0,2,1,2,0,0,...,1,2,1,3,0,1,0,1,e4a775cb,19538871
0,2022-2023,33c895d4,17,0,0,0,1,2,0,1,...,1,1,3,4,1,1,0,1,d07537b9,33c895d4
0,2022-2023,361ca564,17,2,0,2,0,0,1,1,...,1,0,3,2,2,2,2,0,cd051869,361ca564
0,2022-2023,47c64c55,17,0,2,2,0,2,0,0,...,0,2,1,1,0,3,1,1,fd962109,47c64c55
0,2022-2023,4ba7cbea,17,2,0,0,0,0,1,0,...,0,0,0,4,3,2,1,0,cff3d9bb,4ba7cbea
0,2022-2023,5bfb9659,17,0,2,2,0,0,0,1,...,2,0,4,3,1,3,2,1,b8fd03ef,5bfb9659
0,2022-2023,7c21e445,17,0,0,0,2,0,1,1,...,2,0,2,2,1,0,1,0,18bb7c10,18bb7c10
0,2022-2023,822bd0ba,17,2,2,0,0,2,1,0,...,0,1,1,1,2,1,0,0,8602292d,822bd0ba
0,2022-2023,8602292d,17,2,2,0,2,0,0,1,...,4,0,1,1,4,0,3,1,822bd0ba,822bd0ba


### Verificação de NA dados de treino

In [20]:
df_6.isna().sum()

Ano          0
Equipe       0
Rodada       0
TR0          0
TR1          0
TR2          0
TR3          0
TR4          0
TL0          0
TL1          0
TL2          0
TL3          0
TL4          0
TP0          0
TP1          0
TP2          0
TP3          0
TP4          0
TC0          0
TC1          0
TC2          0
TC3          0
TC4          0
TGL         42
Oponente    42
Target      42
GP          42
GC          42
id           0
dtype: int64

### Verificação de NA dados da próxima rodada

In [22]:
df_6_next_round.isna().sum()

Ano         0
Equipe      0
Rodada      0
TR0         0
TR1         0
TR2         0
TR3         0
TR4         0
TL0         0
TL1         0
TL2         0
TL3         0
TL4         0
TP0         0
TP1         0
TP2         0
TP3         0
TP4         0
TC0         0
TC1         0
TC2         0
TC3         0
TC4         0
TGL         0
Oponente    0
id          0
dtype: int64

## Etapa Oponente

### Tratamento

In [23]:
campos = list(df_6)

In [24]:
df_6A = df_6[campos[:campos.index('TGL')+1]]
fields_name = list(df_6A)

In [25]:
df_6B = df_6A.assign(
    **{'TGL': lambda x: x['TGL'].str.replace('0','x').str.replace('1','0').str.replace('x','1')}
).rename(
    columns=dict(
        zip(
            [fields_name[1]]+fields_name[3:]
            , ['Oponente']+['O'+v[1:] if k>2 else v for k, v in enumerate(fields_name)][3:]
        )
    )
)
df_6B

,Ano,Oponente,Rodada,OR0,OR1,OR2,OR3,OR4,OL0,OL1,...,OP1,OP2,OP3,OP4,OC0,OC1,OC2,OC3,OC4,OGL
0,2014-2015,17892952,6,1,0,2,1,0,0,1,...,0,1,1,0,2,1,0,1,1,0
0,2014-2015,17892952,7,2,1,0,2,1,1,0,...,2,0,1,1,0,2,1,0,1,1
0,2014-2015,17892952,8,0,2,1,0,2,0,1,...,1,2,0,1,3,0,2,1,0,0
0,2014-2015,17892952,9,2,0,2,1,0,1,0,...,1,1,2,0,1,3,0,2,1,1
0,2014-2015,17892952,10,0,2,0,2,1,0,1,...,2,1,1,2,1,1,3,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-2023,fd962109,13,2,1,0,0,2,1,1,...,2,1,1,3,0,2,3,4,2,1
0,2022-2023,fd962109,14,2,2,1,0,0,0,1,...,3,2,1,1,2,0,2,3,4,0
0,2022-2023,fd962109,15,1,2,2,1,0,1,0,...,3,3,2,1,0,2,0,2,3,1
0,2022-2023,fd962109,16,0,1,2,2,1,0,1,...,0,3,3,2,2,0,2,0,2,0


### Verificação de NA

In [26]:
df_6B.isna().sum()

Ano          0
Oponente     0
Rodada       0
OR0          0
OR1          0
OR2          0
OR3          0
OR4          0
OL0          0
OL1          0
OL2          0
OL3          0
OL4          0
OP0          0
OP1          0
OP2          0
OP3          0
OP4          0
OC0          0
OC1          0
OC2          0
OC3          0
OC4          0
OGL         42
dtype: int64

## Etapa 7

### Tratamento dos dados de treino

In [34]:
df_7 = df_6.query(
    "TGL.notna()", engine='python'
).set_index(
    ['Ano', 'Oponente', 'Rodada']
).join(
    df_6B.set_index(
        ['Ano', 'Oponente', 'Rodada']
    ).query(
        "OGL.notna()", engine='python'
    )
).reset_index(
)
df_7

,Ano,Oponente,Rodada,Equipe,TR0,TR1,TR2,TR3,TR4,TL0,...,OP1,OP2,OP3,OP4,OC0,OC1,OC2,OC3,OC4,OGL
0,2014-2015,b2b47a98,6,17892952,1,0,2,1,0,0,...,0,3,0,0,2,4,3,0,2,1
1,2014-2015,8ef52968,7,17892952,2,1,0,2,1,1,...,0,2,0,1,0,0,2,1,1,0
2,2014-2015,fb10988f,8,17892952,0,2,1,0,2,0,...,0,0,2,3,2,0,1,4,0,1
3,2014-2015,33c895d4,9,17892952,2,0,2,1,0,1,...,0,2,1,4,0,1,1,0,0,0
4,2014-2015,7c21e445,10,17892952,0,2,0,2,1,0,...,3,2,1,3,1,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5445,2022-2023,8602292d,12,fd962109,1,0,0,2,0,1,...,1,0,1,1,2,1,0,0,1,1
5446,2022-2023,5bfb9659,13,fd962109,2,1,0,0,2,1,...,0,1,0,2,2,1,2,0,5,0
5447,2022-2023,d3fd31cc,14,fd962109,2,2,1,0,0,0,...,0,0,1,2,0,1,2,2,1,1
5448,2022-2023,b8fd03ef,15,fd962109,1,2,2,1,0,1,...,3,0,4,6,0,1,1,0,3,0


### Tratamento dos dados da próxima rodada

In [41]:
df_7_next_round = df_6_next_round.set_index(
    ['Ano', 'Oponente', 'Rodada']
).join(
    df_6B.set_index(
        ['Ano', 'Oponente', 'Rodada']
    )
).reset_index(
).assign(
    **{'OGL': lambda x: x['TGL'].apply(
        lambda y: 0 if y == 1 else 1
    )}
)
df_7_next_round

,Ano,Oponente,Rodada,Equipe,TR0,TR1,TR2,TR3,TR4,TL0,...,OP1,OP2,OP3,OP4,OC0,OC1,OC2,OC3,OC4,OGL
0,2022-2023,7c21e445,17,18bb7c10,2,2,2,1,2,0,...,1,0,2,0,2,2,1,0,1,1
1,2022-2023,e4a775cb,17,19538871,2,0,2,1,2,0,...,2,0,1,0,0,2,5,0,0,1
2,2022-2023,d07537b9,17,33c895d4,0,0,0,1,2,0,...,3,4,1,0,2,2,1,3,0,1
3,2022-2023,cd051869,17,361ca564,2,0,2,0,0,1,...,2,1,0,0,1,2,1,4,0,1
4,2022-2023,fd962109,17,47c64c55,0,2,2,0,2,0,...,1,0,3,3,2,2,0,2,0,1
5,2022-2023,cff3d9bb,17,4ba7cbea,2,0,0,0,0,1,...,0,1,1,0,1,1,4,1,0,1
6,2022-2023,b8fd03ef,17,5bfb9659,0,2,2,0,0,0,...,2,1,3,0,2,1,0,1,1,1
7,2022-2023,18bb7c10,17,7c21e445,0,0,0,2,0,1,...,1,5,1,1,0,0,0,1,0,1
8,2022-2023,8602292d,17,822bd0ba,2,2,0,0,2,1,...,3,0,4,0,1,1,4,0,3,1
9,2022-2023,822bd0ba,17,8602292d,2,2,0,2,0,0,...,2,1,0,1,1,1,2,1,0,1


### Verificação de NA dados de treino

In [42]:
df_7.isna().sum()

Ano         0
Oponente    0
Rodada      0
Equipe      0
TR0         0
TR1         0
TR2         0
TR3         0
TR4         0
TL0         0
TL1         0
TL2         0
TL3         0
TL4         0
TP0         0
TP1         0
TP2         0
TP3         0
TP4         0
TC0         0
TC1         0
TC2         0
TC3         0
TC4         0
TGL         0
Target      0
GP          0
GC          0
id          0
OR0         4
OR1         4
OR2         4
OR3         4
OR4         4
OL0         4
OL1         4
OL2         4
OL3         4
OL4         4
OP0         4
OP1         4
OP2         4
OP3         4
OP4         4
OC0         4
OC1         4
OC2         4
OC3         4
OC4         4
OGL         4
dtype: int64

### Correção dados treino

In [43]:
print(df_7.shape)
df_8 = df_7[df_7['OR0'].notna()]
print(df_8.shape)

(5450, 50)
(5446, 50)


### Verificação de NA dados da próxima rodada

In [44]:
df_7_next_round.isna().sum()

Ano         0
Oponente    0
Rodada      0
Equipe      0
TR0         0
TR1         0
TR2         0
TR3         0
TR4         0
TL0         0
TL1         0
TL2         0
TL3         0
TL4         0
TP0         0
TP1         0
TP2         0
TP3         0
TP4         0
TC0         0
TC1         0
TC2         0
TC3         0
TC4         0
TGL         0
id          0
OR0         0
OR1         0
OR2         0
OR3         0
OR4         0
OL0         0
OL1         0
OL2         0
OL3         0
OL4         0
OP0         0
OP1         0
OP2         0
OP3         0
OP4         0
OC0         0
OC1         0
OC2         0
OC3         0
OC4         0
OGL         0
dtype: int64

### Correção dados da próxima rodada

In [45]:
print(df_7_next_round.shape)
df_8_next_round = df_7_next_round[df_7_next_round['OR0'].notna()]
print(df_8_next_round.shape)

(20, 47)
(20, 47)


### Transformação para type int

In [46]:
for c in list(df_8):
    if c not in ['Ano', 'Oponente', 'Rodada', 'Equipe', 'id']:
        df_8[c] = df_8[c].astype('int')
    
df_8

<ipython-input-46-a5c3278235d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_8[c] = df_8[c].astype('int')


,Ano,Oponente,Rodada,Equipe,TR0,TR1,TR2,TR3,TR4,TL0,...,OP1,OP2,OP3,OP4,OC0,OC1,OC2,OC3,OC4,OGL
0,2014-2015,b2b47a98,6,17892952,1,0,2,1,0,0,...,0,3,0,0,2,4,3,0,2,1
1,2014-2015,8ef52968,7,17892952,2,1,0,2,1,1,...,0,2,0,1,0,0,2,1,1,0
2,2014-2015,fb10988f,8,17892952,0,2,1,0,2,0,...,0,0,2,3,2,0,1,4,0,1
3,2014-2015,33c895d4,9,17892952,2,0,2,1,0,1,...,0,2,1,4,0,1,1,0,0,0
4,2014-2015,7c21e445,10,17892952,0,2,0,2,1,0,...,3,2,1,3,1,1,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5445,2022-2023,8602292d,12,fd962109,1,0,0,2,0,1,...,1,0,1,1,2,1,0,0,1,1
5446,2022-2023,5bfb9659,13,fd962109,2,1,0,0,2,1,...,0,1,0,2,2,1,2,0,5,0
5447,2022-2023,d3fd31cc,14,fd962109,2,2,1,0,0,0,...,0,0,1,2,0,1,2,2,1,1
5448,2022-2023,b8fd03ef,15,fd962109,1,2,2,1,0,1,...,3,0,4,6,0,1,1,0,3,0


In [47]:
for c in list(df_8_next_round):
    if c not in ['Ano', 'Oponente', 'Rodada', 'Equipe', 'id']:
        df_8_next_round[c] = df_8_next_round[c].astype('int')
    
df_8_next_round

,Ano,Oponente,Rodada,Equipe,TR0,TR1,TR2,TR3,TR4,TL0,...,OP1,OP2,OP3,OP4,OC0,OC1,OC2,OC3,OC4,OGL
0,2022-2023,7c21e445,17,18bb7c10,2,2,2,1,2,0,...,1,0,2,0,2,2,1,0,1,1
1,2022-2023,e4a775cb,17,19538871,2,0,2,1,2,0,...,2,0,1,0,0,2,5,0,0,1
2,2022-2023,d07537b9,17,33c895d4,0,0,0,1,2,0,...,3,4,1,0,2,2,1,3,0,1
3,2022-2023,cd051869,17,361ca564,2,0,2,0,0,1,...,2,1,0,0,1,2,1,4,0,1
4,2022-2023,fd962109,17,47c64c55,0,2,2,0,2,0,...,1,0,3,3,2,2,0,2,0,1
5,2022-2023,cff3d9bb,17,4ba7cbea,2,0,0,0,0,1,...,0,1,1,0,1,1,4,1,0,1
6,2022-2023,b8fd03ef,17,5bfb9659,0,2,2,0,0,0,...,2,1,3,0,2,1,0,1,1,1
7,2022-2023,18bb7c10,17,7c21e445,0,0,0,2,0,1,...,1,5,1,1,0,0,0,1,0,1
8,2022-2023,8602292d,17,822bd0ba,2,2,0,0,2,1,...,3,0,4,0,1,1,4,0,3,1
9,2022-2023,822bd0ba,17,8602292d,2,2,0,2,0,0,...,2,1,0,1,1,1,2,1,0,1


## Etapa final salvando dataset

### Dados de treino

In [48]:
df_8[
    ['Ano', 'Rodada', 'id', 'Equipe', 'Oponente']
    + ['TR0', 'TR1', 'TR2', 'TR3', 'TR4', 'TL0', 'TL1', 'TL2', 'TL3', 'TL4', 'TP0', 'TP1', 'TP2', 'TP3', 'TP4', 'TC0', 'TC1', 'TC2', 'TC3', 'TC4', 'TGL']
    + ['OR0', 'OR1', 'OR2', 'OR3', 'OR4', 'OL0', 'OL1', 'OL2', 'OL3', 'OL4', 'OP0', 'OP1', 'OP2', 'OP3', 'OP4', 'OC0', 'OC1', 'OC2', 'OC3', 'OC4', 'OGL']
    + ['Target', 'GP', 'GC']
].to_parquet('dataset_9.parquet')

### Dados da próxima rodada

In [51]:
df_8_next_round[
    ['Ano', 'Rodada', 'id', 'Equipe', 'Oponente']
    + ['TR0', 'TR1', 'TR2', 'TR3', 'TR4', 'TL0', 'TL1', 'TL2', 'TL3', 'TL4', 'TP0', 'TP1', 'TP2', 'TP3', 'TP4', 'TC0', 'TC1', 'TC2', 'TC3', 'TC4', 'TGL']
    + ['OR0', 'OR1', 'OR2', 'OR3', 'OR4', 'OL0', 'OL1', 'OL2', 'OL3', 'OL4', 'OP0', 'OP1', 'OP2', 'OP3', 'OP4', 'OC0', 'OC1', 'OC2', 'OC3', 'OC4', 'OGL']
#     + ['Target', 'GP', 'GC']
].to_parquet('dataset_9_next_round.parquet')